In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import csv

In [2]:
# prendi le recensioni per ogni utente
# dato uno steam id
# è possibile accedere alle recensioni scritte dall'utente tramite la pagina
# https://steamcommunity.com/profiles/{steam_id}/recommended/?p={page_number}
# non tutte le recensioni sono presenti nella stessa pagina
# ma è possibile scorrere le pagine aggiungendo al link
# /?p={page_number}
# (nella prima pagina non è necessario ma si può accedere tranquillamente mettendo p=1)
# il problema è che è possibile accedere anche a pagine dove non sono presenti recensioni
# ma parsando la pagina la lista delle recensioni sarà vuota

In [3]:
def get_reviews(steam_id, page_number):
    url = f"https://steamcommunity.com/profiles/{steam_id}/recommended/?p={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    reviews = soup.findAll('div', {'class': 'review_box'})
    return reviews

In [4]:
with open('output_files/reviews/game_reviews.csv', 'w') as F:
    writer = csv.writer(F)
    cols = ['steam_id', 'app_id', 'recommended', 'hours_played', ]
    writer.writerow(cols)

    steam_ids = [76561199227008553]
    i = 1
    for steam_id in steam_ids:
        while i!=0:
            reviews = get_reviews(steam_id, i)
            if reviews != []:
                for review in reviews:
                    review_data = []
                    review_data.append(str(steam_id))
                    app_id = review.find('a', {'class': 'game_capsule_ctn'})['href'][31:]
                    review_data.append(app_id)
                    recommended = review.find('div', {'class': 'title'}).text
                    review_data.append(recommended)
                    hours_played = review.find('div', {'class': 'hours'}).text
                    hours_played = hours_played.replace('\r', '')
                    hours_played = hours_played.replace('\n', '')
                    hours_played = hours_played.replace('\t', '')
                    review_data.append(hours_played)
                    writer.writerow(review_data)
                i += 1
            else:
                i = 0